In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline    
from nltk.stem import WordNetLemmatizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, classification_report, accuracy_score, f1_score



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv("/kaggle/input/sms-spam-collection-dataset/spam.csv", encoding="latin1")
# problem with utf-8 encoding

In [ ]:
df.head(5)

In [ ]:
df.isna().sum()

In [ ]:
# df_unn2=df[(df['Unnamed:2']!="NaN")]
df_unn2=df.loc[df['Unnamed: 2']!='NaN']
df_unn2

In [ ]:
# df['v2']=re.sub(r'<.*?>|http(s)?://\S+', '',df['v2'])
# df['v2']=df['v2'].str.lower()
# df['v2'] = df['v2'].fillna('missing_value')  # Fill missing values
def data_cleaning(text):
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = str.lower(text) 
    text = text.split()    
    text = ' '.join(text)  
    return text


 
df['Clean Text'] = df['v2'].apply(data_cleaning)

In [ ]:
# unnamed:2, unnamed:3, unnamed:4 dropped
df = df.drop(columns = ["Unnamed: 2","Unnamed: 3","Unnamed: 4"], axis=1)


v1 is the target variable

In [ ]:
df.shape

In [ ]:
df.duplicated().sum()

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
print(df['v1'].value_counts())

In [ ]:
# adding new features: no. of words and no. of sentences
df['No of sentences']=df['v2'].apply(lambda corpus: nltk.sent_tokenize(corpus)).apply(len)
df['No of words']=df['v2'].apply(lambda document: nltk.word_tokenize(document)).apply(len)

In [ ]:
df.head()
df=df.drop(columns=['NO of words'])

In [ ]:
df.describe()

In [ ]:
# sns.scatterplot(x='x',y='y', data=df)
# plt.scatter(df[['No of words','No of sentences']].apply(lambda x:''.join(x),axis=1),df['Numerical Value'])
melted_df = df.melt(id_vars='v1', value_vars=['No of sentences', 'No of words'], var_name='Feature', value_name='Value')

plt.figure(figsize=(15,5))
sns.scatterplot(x='Feature', y='Value', data=melted_df)
plt.show()

In [ ]:
df['Tokenized Text'] = df['Clean Text'].apply(lambda document: nltk.word_tokenize(document))


In [ ]:
def stopwords_removal(tokens):
    stop_words = set(stopwords.words('english'))
    filtered_text = [word for word in tokens if word not in stop_words]
    return filtered_text


# applying stopwords removal
df['Filtered Token'] = df['Tokenized Text'].apply(stopwords_removal)

In [ ]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

In [ ]:
lemmatizer = WordNetLemmatizer()

def lemmatizing(tokens):
    lemmas = [lemmatizer.lemmatize(tok, pos='v') for tok in tokens]
    return lemmas


df['Lemmatized Tokens'] = df['Filtered Token'].apply(lemmatizing)

In [ ]:
corpus = []
for tokens in df['Lemmatized Tokens']:
    document = ' '.join([word for word in tokens])
    corpus.append(document)

In [ ]:
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(corpus).toarray()
y = df['v1']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model=MultinomialNB()

In [ ]:
model.fit(X_train,y_train,sample_weight=None)

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
accuracy=accuracy_score(y_test,y_pred)

In [ ]:
print(accuracy)